In [23]:
import os
fp = '/media/alexn/Data_2/Atlas_Root_Dirs/Duke_Brains/DK17_18/czi_raw_converted/DK17_CFP/'
fn_list = ! ls $fp
for fn in fn_list:
    statinfo = os.stat(fp+fn)
    size_mb = float(statinfo.st_size)/(1024**2)
    if size_mb < 350:
        print(fn)
#         print(size_mb)

DK17_18_slide002_2019_03_10-S3-CFP.tif


In [ ]:
import os
statinfo = os.stat('somefile.txt')
statinfo
(33188, 422511L, 769L, 1, 1032, 100, 926L, 1105022698,1105022732, 1105022732)
statinfo.st_size

In [33]:
! aws s3 cp s3://test-bucket-sid/alex_neuroglancer_volumes/MD585/human_annotation/solid_volume_5um/volume_colored.npy \
/mnt/x/temp_s3_download/volumes/MD585/human_annotation/solid_volume_5um/ 

download: s3://test-bucket-sid/alex_neuroglancer_volumes/MD585/human_annotation/solid_volume_5um/volume_colored.npy to ../../../../../../mnt/x/temp_s3_download/volumes/MD585/human_annotation/solid_volume_5um/volume_colored.npy


In [41]:
! aws s3 cp s3://test-bucket-sid/alex_neuroglancer_volumes/MD589/human_annotation/solid_volume_5um/volume_colored.npy \
/mnt/x/temp_s3_download/volumes/MD589/human_annotation/solid_volume_5um/

download: s3://test-bucket-sid/alex_neuroglancer_volumes/MD589/human_annotation/solid_volume_5um/volume_colored.npy to ../../../../../../mnt/x/temp_s3_download/volumes/MD589/human_annotation/solid_volume_5um/volume_colored.npy


In [25]:
! aws s3 cp s3://test-bucket-sid/alex_neuroglancer_volumes/MD585/human_annotation/solid_volume_5um/volume_colored.npy /mnt/x/temp_s3_download/volumes/MD585/human_annotation/solid_volume_5um/

download failed: s3://test-bucket-sid/alex_neuroglancer_volumes/MD585/human_annotation/solid_volume_5um/volume_colored.npy to ../../../../../../mnt/x/temp_s3_download/volumes/MD585/human_annotation/solid_volume_5um/volume_colored.npy [SSL: DECRYPTION_FAILED_OR_BAD_RECORD_MAC] decryption failed or bad record mac (_ssl.c:2292)


In [1]:
import numpy as np
import pickle as pkl
import boto3

import os

import json
import pathlib
from neuroglancer_scripts.scripts import (generate_scales_info,
                                          slices_to_precomputed,
                                          compute_scales, volume_to_precomputed)
import nibabel as nib
import re

In [28]:
stack = 'MD589'

In [18]:
stack = 'MD585'

In [29]:
s3_creds_file = '/mnt/c/Users/Alex/Documents/json_credentials/alex_aws_credentials.json'
bucket_name = 'test-bucket-sid'
ni_out = '/mnt/x/temp_s3_download/volumes/precomputed/'+stack+'/vol_color.nii'

In [30]:
source_dir = '/mnt/x/temp_s3_download/volumes/'+stack+'/human_annotation/solid_volume_5um/volume_colored.npy'
out_dir = '/mnt/x/temp_s3_download/volumes/precomputed/'+stack+'/'

In [21]:
vol_m = np.load(source_dir)

FileNotFoundError: [Errno 2] No such file or directory: '/mnt/x/temp_s3_download/volumes/MD585/human_annotation/solid_volume_5um/volume_colored.npy'

In [22]:
! mkdir $out_dir

In [13]:
vol_m = np.swapaxes(vol_m,0,2)
print(vol_m.shape)
vol_img = nib.Nifti1Image(vol_m, affine=np.array(\
      [[ 0.005,  0.,  0.,  0.],\
       [ 0.,   0.005,  0.,  0.],\
       [ 0.,  0.,  0.02,  0.],\
       [ 0.,  0.,  0.,  1.]]))
nib.save(vol_img, ni_out)

(1429, 1099, 288)


In [14]:
volume_to_precomputed.main(['', ni_out, out_dir, '--generate-info', '--no-gzip'])

INFO: Input image shape is (1429, 1099, 288)
INFO: Input voxel size is [0.005 0.005 0.02 ] mm
INFO: Detected input axis orientations RAS+
INFO: the following info has been generated:
{
    "type": "image",
    "num_channels": 1,
    "data_type": "uint8",
    "scales": [
        {
            "encoding": "raw",
            "size": [1429, 1099, 288],
            "resolution": [4999.999888241291, 4999.999888241291, 19999.999552965164],
            "voxel_offset": [0, 0, 0]
        }
    ]
}
INFO: The metadata above was written to info_fullres.json. Please run generate-scales-info on that file to generate the 'info' file, then run this program again.
INFO: Neuroglancer transform of the converted volume (written to transform.json):
[[1_0_0_-2499.9999441206455]_[0_1_0_-2499.9999441206455]_[0_0_1_-9999.999776482582]_[0_0_0_1]]


0

In [15]:
with open(os.path.join(os.path.join(out_dir, 'info_fullres.json')), 'r') as info_file:
    info = json.load(info_file)

info["type"] = "segmentation"
# # info["scales"][0]["size"][0] *= 1000000/20000
# # info["scales"][0]["size"][1] *= 1000000/5000
# # info["scales"][0]["size"][2] *= 1000000/5000

# info["scales"][0]["resolution"][0] = 5000
# info["scales"][0]["resolution"][1] = 5000
# info["scales"][0]["resolution"][2] = 20000

with open(os.path.join(os.path.join(out_dir, 'info_fullres.json')), 'w') as info_file:
    json.dump(info, info_file)

# with open(os.path.join(out_dir, 'info_fullres.json'), 'w') as outfile:
#     json.dump(info, outfile)


In [16]:
generate_scales_info.main(['', os.path.join(out_dir, 'info_fullres.json'),  '--encoding' , 'compressed_segmentation',
                           out_dir, '--max-scales', '1', '--target-chunk-size', '128'])

# volume_to_precomputed.main(['', ni_out, out_dir, '--flat', '--no-gzip'])

0

In [17]:
volume_to_precomputed.main(['', ni_out, out_dir, '--flat', '--no-gzip'])

INFO: Loading full volume to memory... 
INFO: Writing chunks... 
writing:  99%|█████████▉| 149/150 [00:41<00:00, 10.86chunks/s]

0

writing: 100%|██████████| 150/150 [01:00<00:00, 10.86chunks/s]

In [31]:
out_dir = '/mnt/x/temp_s3_download/volumes/precomputed/'+stack+'/'
! aws s3 cp --recursive $out_dir s3://test-bucket-sid/final_precomputed_volumes/$stack/

upload: ../../../../../../mnt/x/temp_s3_download/volumes/precomputed/MD589/5um/0-256_1024-1099_256-288 to s3://test-bucket-sid/final_precomputed_volumes/MD589/5um/0-256_1024-1099_256-288
upload: ../../../../../../mnt/x/temp_s3_download/volumes/precomputed/MD589/5um/0-256_1024-1099_128-192 to s3://test-bucket-sid/final_precomputed_volumes/MD589/5um/0-256_1024-1099_128-192
upload: ../../../../../../mnt/x/temp_s3_download/volumes/precomputed/MD589/5um/0-256_1024-1099_0-64 to s3://test-bucket-sid/final_precomputed_volumes/MD589/5um/0-256_1024-1099_0-64
upload: ../../../../../../mnt/x/temp_s3_download/volumes/precomputed/MD589/5um/0-256_1024-1099_64-128 to s3://test-bucket-sid/final_precomputed_volumes/MD589/5um/0-256_1024-1099_64-128
upload: ../../../../../../mnt/x/temp_s3_download/volumes/precomputed/MD589/5um/0-256_1024-1099_192-256 to s3://test-bucket-sid/final_precomputed_volumes/MD589/5um/0-256_1024-1099_192-256
upload: ../../../../../../mnt/x/temp_s3_download/volumes/precomputed/MD58

upload: ../../../../../../mnt/x/temp_s3_download/volumes/precomputed/MD589/5um/256-512_512-768_0-64 to s3://test-bucket-sid/final_precomputed_volumes/MD589/5um/256-512_512-768_0-64
upload: ../../../../../../mnt/x/temp_s3_download/volumes/precomputed/MD589/5um/256-512_768-1024_0-64 to s3://test-bucket-sid/final_precomputed_volumes/MD589/5um/256-512_768-1024_0-64
upload: ../../../../../../mnt/x/temp_s3_download/volumes/precomputed/MD589/5um/1024-1280_768-1024_192-256 to s3://test-bucket-sid/final_precomputed_volumes/MD589/5um/1024-1280_768-1024_192-256
upload: ../../../../../../mnt/x/temp_s3_download/volumes/precomputed/MD589/5um/256-512_768-1024_128-192 to s3://test-bucket-sid/final_precomputed_volumes/MD589/5um/256-512_768-1024_128-192
upload: ../../../../../../mnt/x/temp_s3_download/volumes/precomputed/MD589/5um/256-512_512-768_192-256 to s3://test-bucket-sid/final_precomputed_volumes/MD589/5um/256-512_512-768_192-256
upload: ../../../../../../mnt/x/temp_s3_download/volumes/precompute

upload: ../../../../../../mnt/x/temp_s3_download/volumes/precomputed/MD589/5um/768-1024_256-512_128-192 to s3://test-bucket-sid/final_precomputed_volumes/MD589/5um/768-1024_256-512_128-192
upload: ../../../../../../mnt/x/temp_s3_download/volumes/precomputed/MD589/5um/512-768_768-1024_0-64 to s3://test-bucket-sid/final_precomputed_volumes/MD589/5um/512-768_768-1024_0-64
upload: ../../../../../../mnt/x/temp_s3_download/volumes/precomputed/MD589/5um/768-1024_256-512_64-128 to s3://test-bucket-sid/final_precomputed_volumes/MD589/5um/768-1024_256-512_64-128
upload: ../../../../../../mnt/x/temp_s3_download/volumes/precomputed/MD589/5um/768-1024_256-512_256-288 to s3://test-bucket-sid/final_precomputed_volumes/MD589/5um/768-1024_256-512_256-288
upload: ../../../../../../mnt/x/temp_s3_download/volumes/precomputed/MD589/5um/768-1024_512-768_0-64 to s3://test-bucket-sid/final_precomputed_volumes/MD589/5um/768-1024_512-768_0-64
upload: ../../../../../../mnt/x/temp_s3_download/volumes/precomputed/

In [9]:
def get_bucket(s3_creds_file, bucket_name):
    with open(s3_creds_file) as f:
        creds = json.load(f)
        aws_access_key_id = creds['access_key']
        aws_secret_access_key = creds['secret_key']

    session = boto3.Session(
        aws_access_key_id = aws_access_key_id,
        aws_secret_access_key = aws_secret_access_key,)
    s3 = session.resource('s3')
    return s3.Bucket(bucket_name)


def upload_directory_to_s3(s3_creds_file, bucket_name, dir_to_write_from, s3_dir_to_write_to, overwrite=True):
    bucket = get_bucket(s3_creds_file, bucket_name)
    dir_to_write_from = '{0}'.format(pathlib.Path(dir_to_write_from))
    # write the whole directory onto S3 bucket, upload files
    for subdir, dirs, files in os.walk(dir_to_write_from):
        print('Uploading: {0}'.format(subdir))
        for file in files:
            full_path = os.path.join(subdir, file)
            fp_s3 = re.sub(r'\\', '/', full_path)
            f_key = ''.join([s3_dir_to_write_to, re.sub(re.sub(r'\\', '/', dir_to_write_from), '', fp_s3)])

            if not overwrite:
                obj_keys = [obj.key for obj in bucket.objects.filter(Prefix = f_key)]
                if len(obj_keys) == 0:
                    bucket.upload_file(full_path, f_key)
            else:
                bucket.upload_file(full_path, f_key)